In [ ]:
from omegaconf import OmegaConf

configs = [OmegaConf.load('configs/base_config.yaml')]
configs.append(OmegaConf.load(f'configs/nextqa_grounding.yaml'))
config = OmegaConf.unsafe_merge(*configs)

Batch size:  2


In [4]:
from torch.utils.data import DataLoader
from datasets import get_dataset
from main_batch import my_collate

batch_size = config.dataset.batch_size
dataset = get_dataset(config.dataset)
dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=0,
                        pin_memory=True,
                        collate_fn=my_collate)
input_type = dataset.input_type

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/bill-qian/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /home/bill-
[nltk_data]     qian/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /home/bill-
[nltk_data]     qian/nltk_data...
Warming up PyWSD (takes ~10 secs)... took 2.344268321990967 secs.


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/vu2/nextoe/val.csv'

In [ ]:
from functools import partial
from vision_processes import queues_in, forward

queue_results_main = None
model_name_codex = 'codellama' if config.codex.model == 'codellama' else 'codex'
codex = partial(forward,
                model_name=model_name_codex,
                queues=[queues_in, queue_results_main])

n_batches = len(dataloader)
with open(config.codex.prompt) as f:
    base_prompt = f.read().strip()

In [ ]:
from tqdm import tqdm
from main_batch import run_program

all_results = []
all_answers = []
all_codes = []
all_ids = []
all_queries = []
all_img_paths = []
all_possible_answers = []
all_query_types = []

for i, batch in tqdm(enumerate(dataloader), total=n_batches):

    codes = codex(prompt=batch['query'],
                  base_prompt=base_prompt,
                  input_type=input_type,
                  extra_context=batch['extra_context'])

    # Run the code
    results = []
    for c, sample_id, img, possible_answers, query in \
            zip(codes, batch['sample_id'], batch['image'], batch['possible_answers'], batch['query']):
        result = run_program(
            [c, sample_id, img, possible_answers, query],
            queues_in, input_type)
        # print('Result:', result)
        results.append(result)
    
    all_results += [r[0] for r in results]
    all_codes += [r[1] for r in results]
    all_ids += batch['sample_id']
    all_answers += batch['answer']
    all_possible_answers += batch['possible_answers']
    all_query_types += batch['query_type']
    all_queries += batch['query']
    all_img_paths += [
        dataset.get_sample_path(idx) for idx in batch['index']
    ]

In [ ]:
from rich.syntax import Syntax
from rich.console import Console

console = Console(highlight=False)
for code in all_codes:
    syntax_1 = Syntax(code,
                      "python",
                      theme="monokai",
                      line_numbers=True,
                      start_line=0)
    console.print(syntax_1)

SyntaxError: incomplete input (414409494.py, line 1)